In [ ]:
import json
import os
from pathlib import Path

import git
import numpy as np
from IPython.display import Javascript
from ipywidgets import interact
from munch import Munch
from PIL import Image

repo = git.Repo(Path(".").absolute(), search_parent_directories=True)
ROOT = Path(repo.working_tree_dir)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# Path to the folder where emoji image files are stored
images_path = ROOT / "emoji-images" / "imgs"
# Extract the unicodes from the image file names
image_unicodes = set(p.stem for p in images_path.glob("*.png"))

# Path to the folder where emoji names and unicodes are stored
emojis_path = ROOT / "emojis" / "emojis.json"
# Load the emoji data
with open(emojis_path, "r") as f:
    emojis = json.load(f)["emojis"]

# A dictionary that assigns a name to each unicode with a corresponding image.
unicode2emoji_name = {
    unicode: entry["name"]
    for entry in emojis
    if (unicode := entry["unicode"].replace(" ", "-")) in image_unicodes
}

emoji_name2unicode = {
    unicode2emoji_name[unicode]: unicode for unicode in unicode2emoji_name
}
emoji_names = list(emoji_name2unicode.keys())

In [ ]:
len(emoji_names)

In [ ]:
def draw(objects, canvas_size=(224, 224)):
    canvas = Image.new("RGBA", canvas_size, (0, 0, 0, 255))
    for obj in objects:
        canvas.alpha_composite(obj.image, (obj.bbox.left, obj.bbox.top))
    return canvas


def get_mask(image_array: np.array, threshold=None) -> np.array:
    """Returns the mask of the image. The mask is obtained from the alpha channel.

    :param image_array: An array of shape (H x W x 4) representing an RGBA image.
    :param threshold: determines the threshold for deciding whether to assign a pixel
        to foreground or background. 4 is a good value. Higher values might not for
        example get the correct mask for the emoji "cigarette" (unicode: 1f6ac).
    :returns: The mask of the input image.
    """
    if threshold is None:
        threshold = 4
    mask_array = image_array[:, :, 3]
    mask_array[mask_array < threshold] = 0  # background
    mask_array[mask_array >= threshold] = 255  # foreground
    return mask_array.astype(bool)


def overlap(object1, object2, canvas_size=(224, 224), threshold=0.05):
    """
    :param threshold: the fraction of the areas of any of the two objects that needs
        to be overlapped by the other object.
    """
    canvases = []
    for obj in [object1, object2]:
        canvas = np.zeros(canvas_size, dtype=bool)
        canvas[
            obj.bbox.top : obj.bbox.top + obj.mask.shape[0],
            obj.bbox.left : obj.bbox.left + obj.mask.shape[1],
        ] = obj.mask
        canvases.append(canvas)
    intersection_area = (canvases[0] & canvases[1]).sum()
    return (
        intersection_area / object1.area > threshold
        or intersection_area / object2.area > threshold
    )


def get_bbox(mask_array):
    x_proj = mask_array.any(axis=0)
    y_proj = mask_array.any(axis=1)
    bbox = Munch(
        left=x_proj.argmax(),
        top=y_proj.argmax(),
        right=len(x_proj) - 1 - x_proj[::-1].argmax(),
        bottom=len(y_proj) - 1 - y_proj[::-1].argmax(),
    )
    return bbox


def add_bbox(obj):
    """Adds bounding box to the object."""
    obj.array[:, 0, :3] = 255
    obj.array[:, -1, :3] = 255
    obj.array[0, :, :3] = 255
    obj.array[-1, :, :3] = 255
    obj.array[:, 0, 3] = 255
    obj.array[:, -1, 3] = 255
    obj.array[0, :, 3] = 255
    obj.array[-1, :, 3] = 255
    obj.image = Image.fromarray(obj.array)


def load_emoji(emoji_name, size=(64, 64)):
    unicode = emoji_name2unicode[emoji_name]
    image = Image.open(images_path / (unicode + ".png")).convert("RGBA").resize(size)
    return image

def crop_image(obj):
    """Crops the image of an object and updates affected attributes."""
    bbox = get_bbox(obj.mask)
    obj.array = obj.array[bbox.top : bbox.bottom + 1, bbox.left : bbox.right + 1]
    obj.image = Image.fromarray(obj.array)
    obj.mask = get_mask(obj.array)

In [ ]:
def generate_objects(
    names=["hot dog", "pizza"],
    positions=[(0, 0), (0, 32)],
    rotations=[30, 45],
    orig_image_size=(64, 64),
    crop=False,
    bbox=False,
):
    objects = []
    for name, position, rotation in zip(names, positions, rotations):
        obj = Munch()
        obj.name = name
        obj.rotation = rotation
        obj.image = load_emoji(obj.name, orig_image_size).rotate(obj.rotation)
        obj.array = np.array(obj.image)
        obj.mask = get_mask(obj.array)
        obj.area = obj.mask.sum()
        if crop:
            crop_image(obj)
        obj.bbox = Munch(
            top=position[0],
            left=position[1],
            bottom=position[0] + obj.array.shape[0] - 1,
            right=position[1] + obj.array.shape[1] - 1,
        )
        if bbox:
            add_bbox(obj)
        objects.append(obj)
    return objects

# Relations

In [ ]:
# TODO: Explain the logic behind the two functions.
def point_left_of_directed_line(point, directed_line):
    p = point
    q = directed_line.point
    d = directed_line.vector
    d_rot = np.array([-d[1], d[0]])  # d rotated by 90 degree
    # NOTE: As the y-axis is upside down, the sign is also switched.
    return (p - q) @ d_rot < 0


def point_right_of_directed_line(point, directed_line):
    p = point
    q = directed_line.point
    d = directed_line.vector
    d_rot = np.array([-d[1], d[0]])  # d rotated by 90 degree
    # NOTE: As the y-axis is upside down, the sign is also switched.
    return (p - q) @ d_rot > 0


def test_point_left_of_directed_line():
    point = np.array([1, 1])
    directed_line = Munch(point=np.array([2, -1]), vector=np.array([1, 1]))
    assert point_right_of_directed_line(point, directed_line)
    directed_line = Munch(point=np.array([2, -1]), vector=np.array([-1, 1]))
    assert not point_right_of_directed_line(point, directed_line)


test_point_left_of_directed_line()

In [ ]:
# TODO: Explain the meaning of the variables.
def left_of(object1, object2):
    point_top_right = np.array([object1.bbox.right, object1.bbox.top])
    point_bottom_right = np.array([object1.bbox.right, object1.bbox.bottom])
    directed_line_right = Munch(
        point=np.array([object2.bbox.left, object2.bbox.top]), vector=np.array([-1, -1])
    )
    directed_line_left = Munch(
        point=np.array([object2.bbox.left, object2.bbox.bottom]),
        vector=np.array([-1, 1]),
    )
    constraint1 = point_left_of_directed_line(point_top_right, directed_line_right)
    constraint2 = point_right_of_directed_line(point_bottom_right, directed_line_left)
    # The latter constraint is needed for some boundary cases, where objec1 is much larger than object2.
    return (not overlap(object1, object2)) and (constraint1 and constraint2)


def right_of(object1, object2):
    point_top_left = np.array([object1.bbox.left, object1.bbox.top])
    point_bottom_left = np.array([object1.bbox.left, object1.bbox.bottom])
    directed_line_right = Munch(
        point=np.array([object2.bbox.right, object2.bbox.top]),
        vector=np.array([1, 1]),
    )
    directed_line_left = Munch(
        point=np.array([object2.bbox.right, object2.bbox.bottom]),
        vector=np.array([1, -1]),
    )
    constraint1 = point_left_of_directed_line(point_top_left, directed_line_right)
    constraint2 = point_right_of_directed_line(point_bottom_left, directed_line_left)
    return (not overlap(object1, object2)) and (constraint1 and constraint2)


def above(object1, object2):
    point_bottom_right = np.array([object1.bbox.right, object1.bbox.bottom])
    point_bottom_left = np.array([object1.bbox.left, object1.bbox.bottom])
    directed_line_right = Munch(
        point=np.array([object2.bbox.right, object2.bbox.top]),
        vector=np.array([1, -1]),
    )
    directed_line_left = Munch(
        point=np.array([object2.bbox.left, object2.bbox.top]),
        vector=np.array([-1, -1]),
    )
    constraint1 = point_left_of_directed_line(point_bottom_right, directed_line_right)
    constraint2 = point_right_of_directed_line(point_bottom_left, directed_line_left)
    return (not overlap(object1, object2)) and (constraint1 and constraint2)


def below(object1, object2):
    point_top_right = np.array([object1.bbox.right, object1.bbox.top])
    point_top_left = np.array([object1.bbox.left, object1.bbox.top])
    directed_line_right = Munch(
        point=np.array([object2.bbox.left, object2.bbox.bottom]),
        vector=np.array([-1, 1]),
    )
    directed_line_left = Munch(
        point=np.array([object2.bbox.right, object2.bbox.bottom]),
        vector=np.array([1, 1]),
    )
    constraint1 = point_left_of_directed_line(point_top_left, directed_line_right)
    constraint2 = point_right_of_directed_line(point_top_right, directed_line_left)
    return (not overlap(object1, object2)) and (constraint1 and constraint2)

In [ ]:
@interact(x1=(0, 150), y1=(0, 150), x2=(0, 150), y2=(0, 150))
def test_spatial_relations(x1, y1, x2, y2):
    object1, object2 = generate_objects(
        positions=[(y1, x1), (y2, x2)], orig_image_size=(32, 32), crop=False, bbox=True
    )
    display(draw([object1, object2]))
    print(object1.bbox.top, object1.bbox.bottom, object1.bbox.left, object1.bbox.right)
    print(object2.bbox.top, object2.bbox.bottom, object2.bbox.left, object2.bbox.right)
    print(
        "left_of({}, {}): {}".format(
            object1.name, object2.name, left_of(object1, object2)
        )
    )
    print(
        "right_of({}, {}): {}".format(
            object1.name, object2.name, right_of(object1, object2)
        )
    )
    print(
        "above({}, {}): {}".format(object1.name, object2.name, above(object1, object2))
    )
    print(
        "below({}, {}): {}".format(object1.name, object2.name, below(object1, object2))
    )

In [ ]:
canvas_size = (224, 224)
image_size = (64, 64)
num_objects = 5
positions = list(
    zip(
        np.random.randint(0, canvas_size[0] - image_size[0], (num_objects,)),
        np.random.randint(0, canvas_size[1] - image_size[1], (num_objects,)),
    )
)
names = np.random.choice(emoji_names, size=(num_objects,), replace=False)
rotations = [0] * num_objects
orig_image_size = (32, 32)
print(positions)
print(names)
print(rotations)
objects = generate_objects(
    names=names,
    positions=positions,
    rotations=rotations,
    orig_image_size=orig_image_size,
    crop=False,
    bbox=True,
)
display(draw(objects))
for object1 in objects:
    for object2 in objects:
        if object1 != object2:
            for rel in [left_of, right_of, above, below]:
                print(
                    "{}({}, {}): {}".format(
                        rel.__name__, object1.name, object2.name, rel(object1, object2)
                    )
                )
                print(
                    "{}({}, {}): {}".format(
                        rel.__name__, object2.name, object1.name, rel(object2, object1)
                    )
                )

In [ ]:
from itertools import product

In [ ]:
canvas_size = (224, 224)
image_size = (64, 64)
num_objects = 5
rotations = [0] * num_objects
orig_image_size = (32, 32)
no_overlap = False
while not no_overlap:
    positions = list(
        zip(
            np.random.randint(0, canvas_size[0] - image_size[0], (num_objects,)),
            np.random.randint(0, canvas_size[1] - image_size[1], (num_objects,)),
        )
    )
    names = np.random.choice(emoji_names, size=(num_objects,), replace=False)
#     print(positions)
    print(names)
#     print(rotations)
    objects = generate_objects(
        names=names,
        positions=positions,
        rotations=rotations,
        orig_image_size=orig_image_size,
        crop=False,
        bbox=False,
    )
    resample = False
    for object1, object2 in product(objects, objects):
        if object1 != object2 and overlap(object1, object2):
#             print(object1.name, object2.name, overlap(object1, object2))
            resample = True
            break
#     print("resample", resample)
    if not resample:
        no_overlap = True
display(draw(objects))

# Notification

In [ ]:
Javascript(
    'var snd = new Audio("data:audio/wav;base64,//uQRAAAAWMSLwUIYAAsYkXgoQwAEaYLWfkWgAI0wWs/ItAAAGDgYtAgAyN+QWaAAihwMWm4G8QQRDiMcCBcH3Cc+CDv/7xA4Tvh9Rz/y8QADBwMWgQAZG/ILNAARQ4GLTcDeIIIhxGOBAuD7hOfBB3/94gcJ3w+o5/5eIAIAAAVwWgQAVQ2ORaIQwEMAJiDg95G4nQL7mQVWI6GwRcfsZAcsKkJvxgxEjzFUgfHoSQ9Qq7KNwqHwuB13MA4a1q/DmBrHgPcmjiGoh//EwC5nGPEmS4RcfkVKOhJf+WOgoxJclFz3kgn//dBA+ya1GhurNn8zb//9NNutNuhz31f////9vt///z+IdAEAAAK4LQIAKobHItEIYCGAExBwe8jcToF9zIKrEdDYIuP2MgOWFSE34wYiR5iqQPj0JIeoVdlG4VD4XA67mAcNa1fhzA1jwHuTRxDUQ//iYBczjHiTJcIuPyKlHQkv/LHQUYkuSi57yQT//uggfZNajQ3Vmz+Zt//+mm3Wm3Q576v////+32///5/EOgAAADVghQAAAAA//uQZAUAB1WI0PZugAAAAAoQwAAAEk3nRd2qAAAAACiDgAAAAAAABCqEEQRLCgwpBGMlJkIz8jKhGvj4k6jzRnqasNKIeoh5gI7BJaC1A1AoNBjJgbyApVS4IDlZgDU5WUAxEKDNmmALHzZp0Fkz1FMTmGFl1FMEyodIavcCAUHDWrKAIA4aa2oCgILEBupZgHvAhEBcZ6joQBxS76AgccrFlczBvKLC0QI2cBoCFvfTDAo7eoOQInqDPBtvrDEZBNYN5xwNwxQRfw8ZQ5wQVLvO8OYU+mHvFLlDh05Mdg7BT6YrRPpCBznMB2r//xKJjyyOh+cImr2/4doscwD6neZjuZR4AgAABYAAAABy1xcdQtxYBYYZdifkUDgzzXaXn98Z0oi9ILU5mBjFANmRwlVJ3/6jYDAmxaiDG3/6xjQQCCKkRb/6kg/wW+kSJ5//rLobkLSiKmqP/0ikJuDaSaSf/6JiLYLEYnW/+kXg1WRVJL/9EmQ1YZIsv/6Qzwy5qk7/+tEU0nkls3/zIUMPKNX/6yZLf+kFgAfgGyLFAUwY//uQZAUABcd5UiNPVXAAAApAAAAAE0VZQKw9ISAAACgAAAAAVQIygIElVrFkBS+Jhi+EAuu+lKAkYUEIsmEAEoMeDmCETMvfSHTGkF5RWH7kz/ESHWPAq/kcCRhqBtMdokPdM7vil7RG98A2sc7zO6ZvTdM7pmOUAZTnJW+NXxqmd41dqJ6mLTXxrPpnV8avaIf5SvL7pndPvPpndJR9Kuu8fePvuiuhorgWjp7Mf/PRjxcFCPDkW31srioCExivv9lcwKEaHsf/7ow2Fl1T/9RkXgEhYElAoCLFtMArxwivDJJ+bR1HTKJdlEoTELCIqgEwVGSQ+hIm0NbK8WXcTEI0UPoa2NbG4y2K00JEWbZavJXkYaqo9CRHS55FcZTjKEk3NKoCYUnSQ0rWxrZbFKbKIhOKPZe1cJKzZSaQrIyULHDZmV5K4xySsDRKWOruanGtjLJXFEmwaIbDLX0hIPBUQPVFVkQkDoUNfSoDgQGKPekoxeGzA4DUvnn4bxzcZrtJyipKfPNy5w+9lnXwgqsiyHNeSVpemw4bWb9psYeq//uQZBoABQt4yMVxYAIAAAkQoAAAHvYpL5m6AAgAACXDAAAAD59jblTirQe9upFsmZbpMudy7Lz1X1DYsxOOSWpfPqNX2WqktK0DMvuGwlbNj44TleLPQ+Gsfb+GOWOKJoIrWb3cIMeeON6lz2umTqMXV8Mj30yWPpjoSa9ujK8SyeJP5y5mOW1D6hvLepeveEAEDo0mgCRClOEgANv3B9a6fikgUSu/DmAMATrGx7nng5p5iimPNZsfQLYB2sDLIkzRKZOHGAaUyDcpFBSLG9MCQALgAIgQs2YunOszLSAyQYPVC2YdGGeHD2dTdJk1pAHGAWDjnkcLKFymS3RQZTInzySoBwMG0QueC3gMsCEYxUqlrcxK6k1LQQcsmyYeQPdC2YfuGPASCBkcVMQQqpVJshui1tkXQJQV0OXGAZMXSOEEBRirXbVRQW7ugq7IM7rPWSZyDlM3IuNEkxzCOJ0ny2ThNkyRai1b6ev//3dzNGzNb//4uAvHT5sURcZCFcuKLhOFs8mLAAEAt4UWAAIABAAAAAB4qbHo0tIjVkUU//uQZAwABfSFz3ZqQAAAAAngwAAAE1HjMp2qAAAAACZDgAAAD5UkTE1UgZEUExqYynN1qZvqIOREEFmBcJQkwdxiFtw0qEOkGYfRDifBui9MQg4QAHAqWtAWHoCxu1Yf4VfWLPIM2mHDFsbQEVGwyqQoQcwnfHeIkNt9YnkiaS1oizycqJrx4KOQjahZxWbcZgztj2c49nKmkId44S71j0c8eV9yDK6uPRzx5X18eDvjvQ6yKo9ZSS6l//8elePK/Lf//IInrOF/FvDoADYAGBMGb7FtErm5MXMlmPAJQVgWta7Zx2go+8xJ0UiCb8LHHdftWyLJE0QIAIsI+UbXu67dZMjmgDGCGl1H+vpF4NSDckSIkk7Vd+sxEhBQMRU8j/12UIRhzSaUdQ+rQU5kGeFxm+hb1oh6pWWmv3uvmReDl0UnvtapVaIzo1jZbf/pD6ElLqSX+rUmOQNpJFa/r+sa4e/pBlAABoAAAAA3CUgShLdGIxsY7AUABPRrgCABdDuQ5GC7DqPQCgbbJUAoRSUj+NIEig0YfyWUho1VBBBA//uQZB4ABZx5zfMakeAAAAmwAAAAF5F3P0w9GtAAACfAAAAAwLhMDmAYWMgVEG1U0FIGCBgXBXAtfMH10000EEEEEECUBYln03TTTdNBDZopopYvrTTdNa325mImNg3TTPV9q3pmY0xoO6bv3r00y+IDGid/9aaaZTGMuj9mpu9Mpio1dXrr5HERTZSmqU36A3CumzN/9Robv/Xx4v9ijkSRSNLQhAWumap82WRSBUqXStV/YcS+XVLnSS+WLDroqArFkMEsAS+eWmrUzrO0oEmE40RlMZ5+ODIkAyKAGUwZ3mVKmcamcJnMW26MRPgUw6j+LkhyHGVGYjSUUKNpuJUQoOIAyDvEyG8S5yfK6dhZc0Tx1KI/gviKL6qvvFs1+bWtaz58uUNnryq6kt5RzOCkPWlVqVX2a/EEBUdU1KrXLf40GoiiFXK///qpoiDXrOgqDR38JB0bw7SoL+ZB9o1RCkQjQ2CBYZKd/+VJxZRRZlqSkKiws0WFxUyCwsKiMy7hUVFhIaCrNQsKkTIsLivwKKigsj8XYlwt/WKi2N4d//uQRCSAAjURNIHpMZBGYiaQPSYyAAABLAAAAAAAACWAAAAApUF/Mg+0aohSIRobBAsMlO//Kk4soosy1JSFRYWaLC4qZBYWFRGZdwqKiwkNBVmoWFSJkWFxX4FFRQWR+LsS4W/rFRb/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////VEFHAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAU291bmRib3kuZGUAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAMjAwNGh0dHA6Ly93d3cuc291bmRib3kuZGUAAAAAAAAAACU="); snd.play(); new Notification("Cell Execution Has Finished")'
)

227